In [808]:
import numpy as np
from scipy.signal import argrelextrema
from scipy.signal import find_peaks,detrend
import csv
import math
import matplotlib.pyplot as plt
import glob

In [934]:
def getData(file_name):
    x_arr=[]
    y_arr=[]
    z_arr=[]
    t_arr=[]
    mag_arr=[]
    mean_mag_arr=[]
    actual_steps=None
    with open(file_name, newline='') as csvfile:
        for i,row in enumerate(csvfile):
            row=row.replace("\n","")
            row=row.replace('"',"")
            row = row.split(',')
            if i==3:
                actual_steps=int(row[1])
            if i>=6:
                row=[float(j) for j in row]
                t_arr.append(row[0])
                x_arr.append(row[1])
                y_arr.append(row[2])
                z_arr.append(row[3])  
                mag= math.sqrt(row[1]**2 + row[1]**2 + row[3]**2)
                mag_arr.append(mag)
                mean_mag=sum(mag_arr)/len(mag_arr)
                mean_mag_arr.append(mean_mag)
    return mean_mag_arr,actual_steps


def get_mag_mean_filter_kernel5(file_name):
    actual_steps=None
    t_arr=[]
    x_arr=[]
    y_arr=[]
    z_arr=[]
    mag_arr=[]
    mean_mag_arr=[]
    with open(file_name, newline='') as csvfile:
        for i,row in enumerate(csvfile):
            row=row.replace("\n","")
            row=row.replace('"',"")
            row = row.split(',')
            if i==3:
                actual_steps=int(row[1])
            if i>=6:
                row=[float(j) for j in row]
                t_arr.append(row[0])
                x_arr.append(row[1])
                y_arr.append(row[2])
                z_arr.append(row[3]) 
    x_arr=np.array(x_arr)
    y_arr=np.array(y_arr)
    z_arr=np.array(z_arr)
    mag_arr = np.sqrt(x_arr**2 + y_arr**2 + z_arr**2)          
    mean_mag_arr=mag_arr-np.mean(mag_arr)
    filter_window_size=5
    mag_mean_filter_kernel5 = np.convolve(mean_mag_arr, 
                           np.ones((filter_window_size,))/filter_window_size, 
                           mode='valid')
    return mag_mean_filter_kernel5,actual_steps

In [935]:
def calc_dist(data,actual_steps):
    min_diff=None
    best_dist=None
    calc_peaks=None
    for distance in np.arange(1,10,0.001):
        peaks, _ = find_peaks(data, distance=distance)
        diff=abs(actual_steps-len(peaks))
        if min_diff is None or min_diff>diff:
            min_diff=diff
            best_dist=distance
            calc_peaks=len(peaks)
    return best_dist

def calc_threshold(data,actual_steps):
    min_diff=None
    best_thesh=None
    calc_peaks=None
    for threshold in np.arange(0,1,0.001):
        peaks, _ = find_peaks(data, threshold=threshold)
        diff=abs(actual_steps-len(peaks))
        if min_diff is None or min_diff>diff:
            min_diff=diff
            best_thesh=threshold
            calc_peaks=len(peaks)
    return best_thesh

def calc_peak_dist(data,actual_steps,distance=8.5):
    peaks, _ = find_peaks(data, distance=distance)
    diff=abs(actual_steps-len(peaks))
            #Plotting
#         peak_pos = [data[pos] for pos in peaks]
#     fig = plt.figure(figsize=(15,5))
#     ax = fig.subplots()
#     ax.plot(data)
#     ax.scatter(peaks,peak_pos, color = 'r', s = 15, marker = 'D', label = 'Maxima')
#     ax.legend()
#     ax.grid()
#     plt.show()
    return len(peaks),diff

def calc_peak_thresh(data,actual_steps,threshold=8.5):
    peaks, _ = find_peaks(data, threshold=threshold)
    diff=abs(actual_steps-len(peaks))
            #Plotting
#     peak_pos = [data[pos] for pos in peaks]
#     fig = plt.figure(figsize=(30,15))
#     ax = fig.subplots()
#     ax.plot(data)
#     ax.scatter(peaks,peak_pos, color = 'r', s = 15, marker = 'D', label = 'Maxima')
#     ax.legend()
#     ax.grid()
#     plt.show()
    return len(peaks),diff

In [936]:
def checkWalking(getSignal):
    walking_data = glob.glob("data/walking/*.csv") 
    dist=[]
    thresh=[]
    for file in walking_data:
        data,actual_steps=getSignal(file)
        dist.append(calc_dist(data,actual_steps))
        thresh.append(calc_threshold(data,actual_steps))
    optimal_dist=np.array(dist).mean()
    optimal_thresh=np.array(thresh).mean()
    # print(optimal_dist)
    print(optimal_thresh)
    for file in walking_data:
        data,actual_steps=getSignal(file)
        print("COUNT OF ACTUAL STEPS: ",actual_steps)
#         peaks,diff=calc_peak_dist(data,actual_steps,optimal_dist)
#         print("Difference distance: ",diff)
        peaks,diff=calc_peak_thresh(data,actual_steps,optimal_thresh)
        print("COUNT OF CALCULATED STEPS: ",str(peaks))
        print("Difference threshold: ",diff)
        print("-"*50)

def checkRunning(getSignal):
    running_data = glob.glob("data/running/*.csv") 
    dist=[]
    thresh=[]
    for file in running_data:
        data,actual_steps=getSignal(file)
        dist.append(calc_dist(data,actual_steps))
        thresh.append(calc_threshold(data,actual_steps))
    optimal_dist=np.array(dist).mean()
    optimal_thresh=np.array(thresh).mean()
    # print(optimal_dist)
    # print(optimal_thresh)
    
    for file in running_data:
        data,actual_steps=getSignal(file)
        print("COUNT OF ACTUAL STEPS: ",actual_steps)
#         peaks,diff=calc_peak_dist(data,actual_steps,optimal_dist)
#         print("Difference distance: ",diff)
        peaks,diff=calc_peak_thresh(data,actual_steps,optimal_thresh)
        print("COUNT OF CALCULATED STEPS: ",str(peaks))
        print("Difference threshold: ",diff)
        print("-"*50)

In [937]:
checkWalking(get_mag_mean_filter_kernel5)

0.14040000000000002
COUNT OF ACTUAL STEPS:  103
COUNT OF CALCULATED STEPS:  102
Difference threshold:  1
--------------------------------------------------
COUNT OF ACTUAL STEPS:  113
COUNT OF CALCULATED STEPS:  118
Difference threshold:  5
--------------------------------------------------
COUNT OF ACTUAL STEPS:  100
COUNT OF CALCULATED STEPS:  109
Difference threshold:  9
--------------------------------------------------
COUNT OF ACTUAL STEPS:  116
COUNT OF CALCULATED STEPS:  119
Difference threshold:  3
--------------------------------------------------
COUNT OF ACTUAL STEPS:  103
COUNT OF CALCULATED STEPS:  112
Difference threshold:  9
--------------------------------------------------
COUNT OF ACTUAL STEPS:  112
COUNT OF CALCULATED STEPS:  90
Difference threshold:  22
--------------------------------------------------
COUNT OF ACTUAL STEPS:  100
COUNT OF CALCULATED STEPS:  95
Difference threshold:  5
--------------------------------------------------
COUNT OF ACTUAL STEPS:  115
C

In [938]:
checkRunning(get_mag_mean_filter_kernel5)

COUNT OF ACTUAL STEPS:  163
COUNT OF CALCULATED STEPS:  156
Difference threshold:  7
--------------------------------------------------
COUNT OF ACTUAL STEPS:  160
COUNT OF CALCULATED STEPS:  178
Difference threshold:  18
--------------------------------------------------
COUNT OF ACTUAL STEPS:  157
COUNT OF CALCULATED STEPS:  156
Difference threshold:  1
--------------------------------------------------
COUNT OF ACTUAL STEPS:  163
COUNT OF CALCULATED STEPS:  181
Difference threshold:  18
--------------------------------------------------
COUNT OF ACTUAL STEPS:  160
COUNT OF CALCULATED STEPS:  155
Difference threshold:  5
--------------------------------------------------
COUNT OF ACTUAL STEPS:  163
COUNT OF CALCULATED STEPS:  170
Difference threshold:  7
--------------------------------------------------
COUNT OF ACTUAL STEPS:  164
COUNT OF CALCULATED STEPS:  159
Difference threshold:  5
--------------------------------------------------
COUNT OF ACTUAL STEPS:  166
COUNT OF CALCULATE

In [860]:
x_arr=[]
y_arr=[]
z_arr=[]
t_arr=[]
mag_arr=[]
mean_mag_arr=[]
actual_steps=None
with open("data/walking/walk_1_1.csv", newline='') as csvfile:
    for i,row in enumerate(csvfile):
        row=row.replace("\n","")
        row=row.replace('"',"")
        row = row.split(',')
        if i==3:
            actual_steps=int(row[1])
        if i>=6:
            row=[float(j) for j in row]
            t_arr.append(row[0])
            x_arr.append(row[1])
            y_arr.append(row[2])
            z_arr.append(row[3])  
x_arr=np.array(x_arr)
y_arr=np.array(y_arr)
z_arr=np.array(z_arr)
mag_arr = np.sqrt(x_arr**2 + y_arr**2 + z_arr**2)          
mean_mag_arr=mag_arr-np.mean(mag_arr)
